In [1]:
from stacLib.ard_stac_search import Astac
import rasterio as rio
from rasterio.windows import Window

import sys
import os
import numpy as np
import random
#import tqdm.notebook as tq
import tqdm as tq
import zarr

os.environ['AWS_REQUEST_PAYER']='requester'

In [2]:
bfn = 's3://usgs-landsat-level-3/collection02/DSWE/2021/CU/021/010/LC08_CU_021010_20210412_20210504_02_DSWE/LC08_CU_021010_20210412_20210504_02_INTSM.TIF'

data\landsat_level_3_products\DSWE\h16v04\2021\LC08_CU_016004_20210927_20211004_02_DIAG.tif'

In [3]:
bfn = 's3://usgs-landsat-level-3/collection02/DSWE/2021/CU/021/010/LC08_CU_021010_20210412_20210504_02_DSWE/LC08_CU_016004_20210927_20211004_02_DIAG.tif'

In [4]:
from stacLib.ard_stac_search import Astac

In [5]:
my_date='2021-04-12'

In [6]:
H='21'
V='10'

In [7]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2l3-dswe'
stac_config['description'] = 'not important'

In [8]:
_stac = Astac(stac_config)


https://landsatlook.usgs.gov/stac-server


In [9]:
date_range_text = my_date
cloud_cover_pct_max = '99'
_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

asset_df = _stac.fav_fields_df(_assets)

None 2021-04-12 99


/data/mambaforge/envs/water/lib/python3.11/site-packages/pystac_client/item_search.py:865: FutureWarning: get_all_items_as_dict() is deprecated, use item_collection_as_dict() instead.
  warnings.warn(


1 items found


In [10]:
_assets[0]['assets'].keys()

dict_keys(['index', 'diag', 'intr', 'intsm', 'inwam', 'mask', 'shade', 'xml', 'json'])

In [11]:
my_dswe_intsw_url = _assets[0]['assets']['intsm']['alternate']['s3']['href']

In [12]:
my_dswe_intsw_url

's3://usgs-landsat-level-3/collection02/DSWE/2021/CU/021/010/LC08_CU_021010_20210412_20210504_02_DSWE/LC08_CU_021010_20210412_20210504_02_INTSM.TIF'

In [13]:
my_dswe_diag_url = _assets[0]['assets']['diag']['alternate']['s3']['href']

In [14]:
with rio.open(my_dswe_diag_url ) as dswe:
    dswe_data = dswe.read(1)
    dswe.profile
    print(dswe.count)

1


In [31]:
dswe_data

array([[-9999, -9999, -9999, ...,     0,     0,     0],
       [-9999, -9999, -9999, ...,     0,     0,     0],
       [-9999, -9999, -9999, ...,     0,     0,     0],
       ...,
       [-9999, -9999, -9999, ...,     0,     0,     0],
       [-9999, -9999, -9999, ...,     0,     0,     0],
       [-9999, -9999, -9999, ...,     0,     0,     0]], dtype=int16)

In [33]:
dswe_data.shape

(5000, 5000)

In [15]:
values = np.unique(dswe_data)

In [16]:
values.shape

(26,)

In [17]:
out_data = {}

In [18]:
out_data = np.zeros_like(values, dtype=np.uint8)

In [19]:
out_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint8)

In [20]:
out_data.shape

(26,)

In [21]:
# confident water
out_data[np.where((values == 1111) | (values == 10111) | (values == 11011) | (values == 11101) | (values == 11110) | (values == 11111))] = 1

out_data[np.where((values == 111) | (values == 1011) | (values == 1101) | (values == 1110) | (values == 10011) | (values == 10101) | (values == 10110) | (values == 11001) | (values == 11010) | (values == 11100))] = 2

out_data[np.where(values == 11000)] = 3

out_data[np.where((values == 11) | (values == 101) | (values == 101) | (values == 110) | (values == 1001) | (values == 1010) | (values == 1100) | (values == 10000) | (values == 10001) | (values == 10010) | (values == 10100))] = 4



In [22]:
out_data

array([0, 0, 0, 0, 4, 0, 4, 2, 4, 2, 1, 4, 4, 4, 2, 4, 2, 1, 3, 2, 2, 1,
       2, 1, 1, 1], dtype=uint8)

In [23]:
check = np.unique(out_data)

In [24]:
check.shape

(5,)

In [25]:
check

array([0, 1, 2, 3, 4], dtype=uint8)

In [26]:
out_data = np.where(check == -9999, 0, check)

In [27]:
# shape array with 1 rows and5 columns
array2 = check.reshape(1, 5)

In [28]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = dswe.profile
    

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rio.uint8,
        count=1,
        nodata=0,
        compress='lzw')

    with rio.open('example.tif', 'w', **profile) as dst:
        dst.write(array2.astype(rio.uint8), 1)

# At the end of the ``with rasterio.Env()`` block, context
# manager exits and all drivers are de-registered.

In [29]:
! gdalinfo example.tif

Driver: GTiff/GeoTIFF
Files: example.tif
Size is 5000, 5000
Coordinate System is:
PROJCRS["Albers",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Albers Equal Area",
        METHOD["Albers Equal Area",
            ID["EPSG",9822]],
        PARAMETER["Latitude of false origin",23,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8821]],
        PARAMETER["Longitude of false origin",-96,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8822]],
        PARAMETER["Latitude of 1st standard parallel",29.5,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8823]],
        PARAMETER["Latitude of 2nd standard parallel",45.5,
            ANGLEUNIT["degree",0.0174532925199433],
      

In [35]:
with rio.open('example.tif') as example:
    example_data = example.read(1)
    print(example.count)

1


In [36]:
example_data.shape

(5000, 5000)

In [37]:
example_data

array([[0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       ...,
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4]], dtype=uint8)

In [38]:
import numpy as np

# Your original array of shape (1, 5)
original_array = array2

# Replicate the original array to create a (5000, 5000) array
larger_array = np.tile(original_array, (5000, 5000 // 5))

# If the size is not an exact multiple of 5, you may need to adjust the shape
larger_array = larger_array[:, :5000]

# Now, larger_array is a (5000, 5000) numpy array with the original_array values repeated


In [40]:
# Register GDAL format drivers and configuration options with a
# context manager.
with rio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = dswe.profile
    

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rio.uint8,
        count=1,
        nodata=0,
        compress='lzw')

    with rio.open('larger.tif', 'w', **profile) as dst:
        dst.write(larger_array.astype(rio.uint8), 1)

# At the end of the ``with rasterio.Env()`` block, context
# manager exits and all drivers are de-registered.

In [41]:
with rio.open('larger.tif') as example:
    example_data = example.read(1)
    print(example.count)

1


In [42]:
example_data

array([[0, 1, 2, ..., 2, 3, 4],
       [0, 1, 2, ..., 2, 3, 4],
       [0, 1, 2, ..., 2, 3, 4],
       ...,
       [0, 1, 2, ..., 2, 3, 4],
       [0, 1, 2, ..., 2, 3, 4],
       [0, 1, 2, ..., 2, 3, 4]], dtype=uint8)

In [43]:
!gdalinfo larger.tif

Driver: GTiff/GeoTIFF
Files: larger.tif
Size is 5000, 5000
Coordinate System is:
PROJCRS["Albers",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["Albers Equal Area",
        METHOD["Albers Equal Area",
            ID["EPSG",9822]],
        PARAMETER["Latitude of false origin",23,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8821]],
        PARAMETER["Longitude of false origin",-96,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8822]],
        PARAMETER["Latitude of 1st standard parallel",29.5,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8823]],
        PARAMETER["Latitude of 2nd standard parallel",45.5,
            ANGLEUNIT["degree",0.0174532925199433],
       